In [11]:
import pandas as pd
import os
import math
from scipy import stats as st
import statsmodels.stats.multitest as ssm

from rp2 import notebooks, paths
from evolution import Load_data

nb = notebooks.initialise_environment("Pairwise_Slope_Comparison",
                                      dependencies=["Mean_Var_Linear_Regression"],
)

Create dataframe containing the slopes and the standard error (se) of the slopes obtained by robust linear regression fitted on mean and variance of the mRNA counts.

In [2]:
mouse_trends_df = Load_data.load_trends_good_fits("mouse")
rat_trends_df = Load_data.load_trends_good_fits("rat") 
pig_trends_df = Load_data.load_trends_good_fits("pig")
rabbit_trends_df = Load_data.load_trends_good_fits("rabbit")

mouse_trends_sorted_df = mouse_trends_df.sort_values(by=["gene"])
rat_trends_sorted_df = rat_trends_df.sort_values(by = ["mouse_id"])
pig_trends_sorted_df = pig_trends_df.sort_values(by = ["mouse_id"])
rabbit_trends_sorted_df = rabbit_trends_df.sort_values(by = ["mouse_id"])

rat_ordered_slopes = []
for slope in rat_trends_sorted_df.slope:
    rat_ordered_slopes.append(slope)
    
rat_ordered_se = []
for error in rat_trends_sorted_df.slope_error:
    rat_ordered_se.append(error)
    
pig_ordered_slopes = []
for slope in pig_trends_sorted_df.slope:
    pig_ordered_slopes.append(slope)
    
pig_ordered_se = []
for error in pig_trends_sorted_df.slope_error:
    pig_ordered_se.append(error)
    
rabbit_ordered_slopes = []
for slope in rabbit_trends_sorted_df.slope:
    rabbit_ordered_slopes.append(slope)
    
rabbit_ordered_se = []
for error in rabbit_trends_sorted_df.slope_error:
    rabbit_ordered_se.append(error)

compare_slopes_df = pd.DataFrame(mouse_trends_sorted_df.gene, columns = ["gene"])
compare_slopes_df ["mouse_slope"] = mouse_trends_sorted_df.slope
compare_slopes_df ["mouse_slope_se"] = mouse_trends_sorted_df.slope_error

compare_slopes_df ["rat_slope"] = rat_ordered_slopes
compare_slopes_df ["rat_slope_se"] = rat_ordered_se

compare_slopes_df ["pig_slope"] = pig_ordered_slopes
compare_slopes_df ["pig_slope_se"] = pig_ordered_se

compare_slopes_df ["rabbit_slope"] = rabbit_ordered_slopes
compare_slopes_df ["rabbit_slope_se"] = rabbit_ordered_se

compare_slopes_df

,gene,mouse_slope,mouse_slope_se,rat_slope,rat_slope_se,pig_slope,pig_slope_se,rabbit_slope,rabbit_slope_se
0,ENSMUSG00000000253,1.342178,0.160848,1.465687,0.076001,1.648244,0.053422,0.997366,0.063941
1,ENSMUSG00000000275,2.962519,0.167625,1.236174,0.033765,1.084045,0.079700,0.747997,0.126128
2,ENSMUSG00000000805,4.619336,0.173514,20.613551,0.146169,12.081549,1.121550,2.719112,0.085236
3,ENSMUSG00000000838,2.215212,0.326673,1.280698,0.114172,1.511315,0.274940,0.646330,0.112090
4,ENSMUSG00000000982,33.856494,0.615989,92.565669,2.224853,183.980384,10.122569,41.891629,4.785654
...,...,...,...,...,...,...,...,...,...
164,ENSMUSG00000071654,1.344678,0.197864,1.658202,0.063236,1.385738,0.043343,1.226611,0.142908
165,ENSMUSG00000072872,2.036422,0.239236,1.373524,0.272969,1.946681,0.110132,1.360331,0.122686
166,ENSMUSG00000074622,3.539442,0.206792,4.718845,0.801614,3.680105,0.208498,1.669179,0.251235
167,ENSMUSG00000074794,2.364267,0.046652,1.426201,0.021390,1.922775,0.109290,2.027745,0.116107


Number of individual data points used to fit the mean-variance lines in each species:

In [3]:
n_values = {
    "n_mouse": 20,
    "n_rat": 21,
    "n_pig": 12,
    "n_rabbit": 12
}

Pairwise comparison of the slopes by t-test:

In [4]:
def regression_slopes_t_test(number, species1, species2):
    slope1 = compare_slopes_df.loc[number, f"{species1}_slope"]
    slope2 = compare_slopes_df.loc[number, f"{species2}_slope"]
    std_error1 = compare_slopes_df.loc[number, f"{species1}_slope_se"]
    std_error2 = compare_slopes_df.loc[number, f"{species2}_slope_se"]
    
    numerator = slope1-slope2
    denominator = math.sqrt((std_error1**2)+(std_error2**2))
    t_statistic = numerator/denominator
    
    n1 = n_values[f"n_{species1}"]
    n2 = n_values[f"n_{species2}"]
    degree_of_freedom = n1 + n2 - 4
    
    return t_statistic, degree_of_freedom

def calculate_p_value(t_statistic, d_o_f):
    p_value = (1 - st.t.cdf(abs(t_statistic), df=d_o_f))*2
    return p_value

In [5]:
species = ["mouse", "rat", "pig", "rabbit"]
all_pairs = [(species_1, species_2) for i, species_1 in enumerate(species) for species_2 in species[i +1: ]]
print(all_pairs)

[('mouse', 'rat'), ('mouse', 'pig'), ('mouse', 'rabbit'), ('rat', 'pig'), ('rat', 'rabbit'), ('pig', 'rabbit')]


In [7]:
a = 0

gene_list = []
species_1_list = []
species_2_list = []
t_stat_list = []
degree_of_freedom_list = []
p_val_list = []

for gene in compare_slopes_df.gene:
    
    for pair in all_pairs:
        species_1 = pair[0]
        species_2 = pair[1]
    
        t_statistic, d_o_f = regression_slopes_t_test(a, species_1, species_2)
        p_val = calculate_p_value (t_statistic, d_o_f)
        
        gene_list.append(gene)
        species_1_list.append(species_1)
        species_2_list.append(species_2)
        t_stat_list.append(t_statistic)
        degree_of_freedom_list.append(d_o_f)
        p_val_list.append(p_val)
   
    a += 1


In [8]:
regression_results_df = pd.DataFrame(gene_list, columns = ["gene"])
regression_results_df ["species_1"] = species_1_list
regression_results_df ["species_2"] = species_2_list
regression_results_df ["t_statistic"] = t_stat_list
regression_results_df ["degree_of_freedom"] = degree_of_freedom_list
regression_results_df ["p_value"] = p_val_list

regression_results_df

,gene,species_1,species_2,t_statistic,degree_of_freedom,p_value
0,ENSMUSG00000000253,mouse,rat,-0.694259,37,4.918565e-01
1,ENSMUSG00000000253,mouse,pig,-1.805824,28,8.170808e-02
2,ENSMUSG00000000253,mouse,rabbit,1.992080,28,5.619515e-02
3,ENSMUSG00000000253,rat,pig,-1.965134,29,5.904221e-02
4,ENSMUSG00000000253,rat,rabbit,4.715247,29,5.594360e-05
...,...,...,...,...,...,...
1009,ENSMUSG00000093930,mouse,pig,1.072332,28,2.927267e-01
1010,ENSMUSG00000093930,mouse,rabbit,5.048198,28,2.430920e-05
1011,ENSMUSG00000093930,rat,pig,-4.852369,29,3.823239e-05
1012,ENSMUSG00000093930,rat,rabbit,3.181757,29,3.476586e-03


Perform Benjamini-Hochberg correction on p-values to account for multiple testing.

In [13]:
def BH_correction(p_values):
    FDR_results = ssm.multipletests(p_values, 0.05, method="fdr_bh")
    results = {
        "FDR_values" : FDR_results[1],
        "significance" : FDR_results[0]
    }
    
    return pd.Series(results)

In [14]:
BH_correction_results = BH_correction(p_val_list)

In [15]:
regression_results_df ["FDR"] = BH_correction_results.FDR_values
regression_results_df ["p_significance"] = BH_correction_results.significance

regression_results_df

,gene,species_1,species_2,t_statistic,degree_of_freedom,p_value,FDR,p_significance
0,ENSMUSG00000000253,mouse,rat,-0.694259,37,4.918565e-01,5.585023e-01,False
1,ENSMUSG00000000253,mouse,pig,-1.805824,28,8.170808e-02,1.155537e-01,False
2,ENSMUSG00000000253,mouse,rabbit,1.992080,28,5.619515e-02,8.306397e-02,False
3,ENSMUSG00000000253,rat,pig,-1.965134,29,5.904221e-02,8.674456e-02,False
4,ENSMUSG00000000253,rat,rabbit,4.715247,29,5.594360e-05,1.500709e-04,True
...,...,...,...,...,...,...,...,...
1009,ENSMUSG00000093930,mouse,pig,1.072332,28,2.927267e-01,3.550536e-01,False
1010,ENSMUSG00000093930,mouse,rabbit,5.048198,28,2.430920e-05,6.828125e-05,True
1011,ENSMUSG00000093930,rat,pig,-4.852369,29,3.823239e-05,1.053469e-04,True
1012,ENSMUSG00000093930,rat,rabbit,3.181757,29,3.476586e-03,6.676626e-03,True


In [16]:
output_path = paths.get_output_path()

if not os.path.exists(f"{output_path}\\Pairwise_comparison"):
    os.makedirs(f"{output_path}\\Pairwise_comparison")

filename = "pairwise_comparison_FDR_results.xlsx"
path = str(paths.get_output_path("Pairwise_comparison")) + "\\" + filename

with pd.ExcelWriter(path) as writer:
    regression_results_df.to_excel(writer)
    
print("Output file created here:", path)

Output file created here: C:\Users\wolke\Documents\Code\Output\Pairwise_comparison\pairwise_comparison_FDR_results.xlsx


Determine the number of significant FDR values obtained for each gene (0-6).

In [17]:
p_numbers = {}

row = 0
for gene in regression_results_df.gene:
    
    p_sig = regression_results_df.iloc[row, 7]
    
    if gene not in p_numbers.keys():
        if p_sig == True:
            p_numbers[gene] = 1
            
        if p_sig == False:
            p_numbers[gene] = 0
    else:
        if p_sig == True:
            p_numbers[gene] +=1
            
    row +=1

gene_list = p_numbers.keys()
significant_p_list = p_numbers.values()

In [20]:
significant_p_val_df = pd.DataFrame(gene_list, columns = ["gene"])
significant_p_val_df ["symbol"] = mouse_trends_df.symbol
significant_p_val_df ["significant_p"] = significant_p_list

significant_p_val_df

,gene,symbol,significant_p
0,ENSMUSG00000000253,Gmpr,2
1,ENSMUSG00000000275,Trim25,4
2,ENSMUSG00000000805,Car4,6
3,ENSMUSG00000000838,Fmr1,4
4,ENSMUSG00000000982,Ccl3,5
...,...,...,...
164,ENSMUSG00000071654,Uqcc3,2
165,ENSMUSG00000072872,Rybp,2
166,ENSMUSG00000074622,Mafb,3
167,ENSMUSG00000074794,Arrdc3,5


In [21]:
filename = "pairwise_comparison_number_of_significant_FDR.xlsx"
path = str(paths.get_output_path("Pairwise_comparison")) + "\\" + filename

with pd.ExcelWriter(path) as writer:
    significant_p_val_df.to_excel(writer)
    
print("Output file created here:", path)

Output file created here: C:\Users\wolke\Documents\Code\Output\Pairwise_comparison\pairwise_comparison_number_of_significant_FDR.xlsx
